# Import Libraries

In [1]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import layers

2022-07-20 09:47:25.192081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 09:47:25.192098: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load Dataset

Use `with_info=True` to get the metadata about the dataset and `as_supervised=True` to receive the corresponding labels on the dataset. The `tf_flowers` dataset does not include a test set, so our only options are from the training set. 

In [6]:
(training_set, validation_set), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
    )

In [16]:
print(f"Length of training set:{len(training_set)}, and length of validation set:{len(validation_set)}")

Length of training set:2936, and length of validation set:734


## Resizing the Images

We need to resize the images as our model expects images all of the same size. The ".resize" method Resizes `images` to `size` using the specified `method`. Then we normalize all the pixel values as all tensorflorhub models expect tensors to be in the range [0,1]

In [3]:
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label

For this model we will be transferring over the MobileNet model which expects images of size 224 x 224. 

In [4]:
num_examples = dataset_info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

### Input Pipeline

Because we don't care about the order in which the images are resized we can do them "in parallel" and `AUTOTUNE` decides the number of images done in parallel. We `.cache()` the batches to speed up future execution of the pre-processing. The shuffle buffer size just ensures the shuffle the data randomly. The `.prefetch(AUTOTUNE)` will prefetch 32 examples instantly after the previous GPU calls are finished

In [7]:
# Tensorflow deciding AUTOTUNE value
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_batches = training_set.cache().shuffle(num_examples//4).map(format_image,num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

validation_batches = validation_set.cache().map(format_image,num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)


# Transfer Learning with MobileNet

First get the URL from TensorFlowHub for the specific model we are using

In [9]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feauture_extractor = hub.KerasLayer(URL, input_shape=(IMAGE_RES, IMAGE_RES,3))

Freeze all the variables within the feature_extractor layer so that during training none of the parameters are changed within these layers

In [10]:
feauture_extractor.trainable = False

## Attaching a Classification Final Layer

In [19]:
# Find the number of different classes in the flowers dataset
classes_num = dataset_info.features['label'].num_classes
classes_num

5

We need to wrap the hub layer with a tf.keras.Sequential model and add the new classification layer

In [20]:
model = tf.keras.Sequential([
    feauture_extractor,
    layers.Dense(classes_num, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


## Training the Model

In [21]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

EPOCHS = 5
history = model.fit(train_batches,
                   epochs=EPOCHS,
                   validation_data=validation_batches)

Epoch 1/5
92/92 [==============================] - 33s 338ms/step - loss: 0.6973 - accuracy: 0.7483 - val_loss: 0.4184 - val_accuracy: 0.8638
Epoch 2/5
92/92 [==============================] - 29s 319ms/step - loss: 0.3640 - accuracy: 0.8770 - val_loss: 0.3386 - val_accuracy: 0.8896
Epoch 3/5
92/92 [==============================] - 23s 255ms/step - loss: 0.2899 - accuracy: 0.9040 - val_loss: 0.2983 - val_accuracy: 0.9114
Epoch 4/5
92/92 [==============================] - 22s 238ms/step - loss: 0.2379 - accuracy: 0.9268 - val_loss: 0.2754 - val_accuracy: 0.9169
Epoch 5/5
92/92 [==============================] - 22s 243ms/step - loss: 0.2049 - accuracy: 0.9435 - val_loss: 0.2717 - val_accuracy: 0.9183


## Checking Predictions

In [23]:
class_names = np.array(dataset_info.features['label'].names)
class_names

array(['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'],
      dtype='<U10')

Run an image batch through the model and convert the indices to class names

In [30]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
label_batch = label_batch.numpy()


predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = class_names[predicted_ids]

predicted_class_names

1/1 [==============================] - 0s 215ms/step


array(['tulips', 'tulips', 'sunflowers', 'dandelion', 'sunflowers',
       'roses', 'daisy', 'roses', 'dandelion', 'daisy', 'daisy',
       'sunflowers', 'dandelion', 'dandelion', 'dandelion', 'sunflowers',
       'daisy', 'dandelion', 'dandelion', 'roses', 'tulips', 'sunflowers',
       'dandelion', 'dandelion', 'daisy', 'roses', 'sunflowers', 'daisy',
       'daisy', 'sunflowers', 'tulips', 'daisy'], dtype='<U10')

Now compare with the true labels and predicted ones

In [31]:
print("Labels:           ", label_batch)
print("Predicted Labels: ",predicted_ids)

Labels:            [2 2 3 0 3 4 1 2 0 1 1 3 0 0 0 3 1 0 0 4 2 3 0 0 1 2 3 1 1 3 2 1]
Predicted Labels:  [2 2 3 0 3 4 1 4 0 1 1 3 0 0 0 3 1 0 0 4 2 3 0 0 1 4 3 1 1 3 2 1]


# Saving Our Model

Once the model has been trained, we can save the model as a HDF5 file which is the format used by Keras, and this file will have the extension `.h5` and its name will correspond to the current time stamp

In [32]:
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1658338888.h5


In [37]:
!ls -lh | grep .h5

-rw-rw-r-- 1 alex alex 9.0M Jul 20 10:41 1658338888.h5


# Export as SavedModel

Another way to save models is to use the TensorFlow SavedModel format which is a standalone serialization format for Tensorflow objects. It contains a complete TensorFlow program, including weights and computations and it doesn't require the original model building code to run.

The SavedModel files that are created:
1. tf checkpoint containing model weights
2. A SavedModel proto containing the underlying tf graph. Separate graphs are saved for predictions (serving), train, and evaluation. If the model wasn't compiled before, then only the inference graph gets exported
3. The model's architecture config, if available

The `tf.saved_model.save` method takes in the original model we desire to save and the path to the folder where we want to save the model. This function will create a folder where you'll find an `assets` folder, a `variables` folder, and the `saved_model.pb` folder